Testando se o ambiente está funcionando

In [ ]:
print('teste')

Importando bibliotecas para explorar os diretórios do computador

In [ ]:
from os import listdir
from os.path import isfile, join

Verificando todos os arquivos obtidos da fonta de dados

In [ ]:
path = "E://dump"

dump_files = listdir(path)

In [ ]:
import pandas as pd
import numpy as np

Tendo certeza que é possível ler os dados do GHtorrent.

In [ ]:
#pd.read_csv(join(path, 'project_languages.csv'))

Acredito que será possível partir dos pullrequest para observar quais repositórios possuem contribuições ou não. Além disso, após pesquisar (https://levelup.gitconnected.com/how-to-mine-github-data-in-2022-e9c70b3f61d3) parece que os dados de 2015 possuem uma estrutura diferente. Então irei tentar filtrar eles fora do meu dataset

In [ ]:
#df_pullrequests = pd.read_csv(join(path, 'pull_requests.csv'))

In [ ]:
#df_pullrequests

Um artigo que mostra a estrutura dos arquivos do GHtorrent https://gousios.org/pub/ghtorrent-dataset-toolsuite.pdf
misturando com esse aqui também https://github.com/ghtorrent/ghtorrent.org/blob/master/files/schema.pdf
Então irei adicionar o nome nas colunas, acabei retirando a coluna pullrequest_id pois ela não fazia sentido no contexto

In [ ]:
#df_pullrequests.columns = ["id", "head_repo_id", "base_repo_id", "head_commit_id", "base_commit_id", "intra_branch_id","merged"]

Como os pull requests não possuem data de criação tentarei filtrar através dos commits que eles estão ligados.

In [ ]:
# # Função para calcular o número total de chunks
# def get_total_chunks(filename, chunksize):
#     # Contar o número total de linhas no arquivo
#     total_lines = sum(1 for _ in open(filename)) - 1  # Subtrai 1 por causa do cabeçalho
#     # Calcular o número total de chunks
#     total_chunks = (total_lines // chunksize) + (1 if total_lines % chunksize != 0 else 0)
#     return total_chunks

# # Exemplo de uso
# filename = join(path, 'commits.csv')
# chunksize = 5000000
# total_chunks = get_total_chunks(filename, chunksize)
# print(f"Total de chunks: {total_chunks}")

In [ ]:
#df_commits_19 = pd.DataFrame({},columns=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"])



In [ ]:
#def checkDate(df):
#   return pd.to_datetime(df.iloc[0]['created_at']) > pd.to_datetime('2019-01-01') or pd.to_datetime(df.iloc[-1]['created_at']) > pd.to_datetime('2019-01-01')




In [ ]:
# for chunk in df_commits:
#     chunk.map(lambda row: chechRowDate(row))

In [ ]:
# i=0

# for chunk in df_commits:
#     if checkDate(chunk):
#         chunk.to_csv(f'E:/dump/commits_greater_than_2k19_{i}')

#     i += 1

# Filtrando os repositórios Open Source

Devido a grande quantidade de commits, não foi possível os utilizar como filtro de projetos open-source. Nesse caso, iremos utilizar a relação project_members para encontrar quais projetos são relevantes, ou seja, que possuem contribuintes além do dono.

In [ ]:
df_project_members = pd.read_csv(join(path, 'project_members.csv'), names=["repo_id", "user_id", "created_at", "ext_ref_id"])

In [ ]:
df_project_members

In [ ]:
df_count_members = df_project_members.groupby("repo_id", as_index=False)["user_id"].count()

In [ ]:
df_count_members

In [ ]:
df_open_source_members = df_count_members[df_count_members["user_id"]>10]

In [ ]:
df_open_source_members

Devivo a datação dos dados iremos filtrar somente os membros a partir de 2019

In [ ]:
df_project_members = df_project_members.loc[
    (pd.to_datetime(df_project_members['created_at']) > pd.to_datetime("01-01-2019")) & 
    (df_project_members['repo_id'].isin(df_open_source_members['repo_id']))
    ]

In [ ]:
#transforma o campo de data em data

df_project_members['created_at'] = pd.to_datetime(df_project_members['created_at'])

In [ ]:
df_project_members

# Dataframe alvo

A partir do estudo Recommending GitHub Projects for Developer Onboarding podemos extrair alguns atributos a serem extraídos do dataset para que seja estudado o quão receptivo um repositório é, são eles:

- Crescimento do Projeto: Quantidade de commits do projeto no momento de iniciação
- Data de criação do projeto: Projetos mais novos tendem a atrair mais desenvolvedores
- Momento de primeira contribuição: quando o projeto começa a receber contribuidores
- Momento da última contribuição: se a equipe do projeto se manter estável significa que existe menos oportunidades de onboarding
- Momento do primeiro commit: primeiro commit do projeto, a partir de quando passa a aceitar contribuições
- Momento do último commit: se o projeto receber muitos commits, mas nao tiver tanto onboarding significa q esse projeto necessita de mais contribuintes

In [ ]:
#objetivo final

df_onboardings = pd.DataFrame({}, columns=["repo_id", "created_at", "total_commits","first_onboarding", "last_onboarding", "first_commit", "last_commit", "user_id", "onboarding_date"])

df_onboardings

In [ ]:
chunks_commits = pd.read_csv(join(path, 'commits.csv'), names=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"], chunksize=5000000)

Agora irei filtrar somente os commits dos projetos selecionados como open source após o ano de 2019

# Pegando os commits referentes aos repositórios classificados como Open Source

In [ ]:

#i=0
#for chunk in chunks_commits:
#        df_commit = chunk[chunk['project_id'].isin(df_project_members['repo_id'])]
#        df_commit.to_csv(f'E:/dump/open_source/open_source_commits_{i}.csv')
#        i += 1


In [ ]:
dump_open_source_files = listdir(join(path, 'open_source'))


# Descobrindo usuários iniciantes

Para determinar quais contribuições são referentes a iniciantes teremos que descobrir qual foi o primeiro commit de cada usuário e qual foi o repositório dele

In [ ]:
first_user_commit_by_chunks = []

i=0
for file in dump_open_source_files:
    print("lendo arquivo: ", i)
    df_commit = pd.read_csv(join(path,'open_source',file))
    df_commit["created_at"] = pd.to_datetime(df_commit["created_at"])
    first_user_commit_by_chunks.append( df_commit.groupby(["author_id"], as_index=False).agg({"created_at": np.min}))
    i += 1

In [ ]:
df_first_user_commit = pd.concat(first_user_commit_by_chunks)

#df_first_user_commit = df_first_user_commit.groupby("author_id", as_index=False).agg({"created_at": np.min})

df_first_user_commit

testando um método de filtrar os primeiros commits em repositórios open source de cada usuário sem q o ID e o repo_id sejam abstraídos

In [ ]:
def get_min_date_row(group):
    min_row = group.loc[group['created_at'].idxmin()]
    return min_row



In [ ]:
first_user_commit_by_chunks2 = []

i=0
for file in dump_open_source_files:
    print("lendo arquivo: ", i)
    df_commit = pd.read_csv(join(path,'open_source',file))
    df_commit["created_at"] = pd.to_datetime(df_commit["created_at"])
    first_user_commit_by_chunks2.append(df_commit.groupby('author_id').apply(get_min_date_row).reset_index(drop=True))
    i += 1

In [ ]:
first_user_commit_by_chunks2

In [ ]:
df_first_user_commit2 = pd.concat(first_user_commit_by_chunks2)

df_first_user_contrib = df_first_user_commit2.groupby('author_id').apply(get_min_date_row).reset_index(drop=True)

df_first_user_contrib

In [ ]:
df_first_user_contrib

filtrando somente as contribuições posteriores a 2019

In [ ]:
df_first_user_commit = df_first_user_commit.loc[df_first_user_commit['created_at'] > pd.to_datetime('2019-01-01')]

Agora calculamos quantos commits cada projeto possui no momento de onboard do dev

In [ ]:

def get_total_commits(row):
    repo_id = row['project_id']
    date = row['created_at']
    total_commits = 0
    for file in dump_open_source_files:
        chunk = pd.read_csv(dump_open_source_files)
        total_commits += chunk.loc[(chunk['project_id'] == repo_id) & (pd.to_datetime(chunk['created_at']) <= pd.to_datetime(date))].shape[0]

    return total_commits

In [ ]:

['total_commits'] = df_first_commit_user.apply(get_total_commits, axis=1)   

Pegar os detalhes do projeto

In [ ]:
df_projetos = pd.read_csv(join(path, "projects.csv"), names=["id", "url", "owner_id", "name", "descriptor", "language", "created_at", "forked_from", "deleted", "updated_at"], na_values="not available", on_bad_lines='warn', chunksize=5000000)